In [ ]:
import pandas  as pd
import glob,os
import matplotlib.pyplot as plt
import matplotlib.style as style
import warnings
from datetime import datetime
import numpy as np
import seaborn as sns

In [ ]:
#配置
warnings.filterwarnings("ignore")
pd.set_option("display.float_format","{:.4f}".format)
style.use("ggplot")
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams["figure.dpi"] = 200

In [ ]:
%pwd

In [46]:
os.chdir("/home/jovyan/work/data/驱虫剂市场")  

In [47]:
files = glob.glob("./*交易额.xlsx")

In [ ]:
dfs = []
for f in files:
    t = f.split("市场")[0][2:]  #前缀
    df = pd.read_excel(f)
    
    if df.时间.dtype == "int64":
        df["时间"] = pd.to_datetime(df["时间"],unit="D",origin=pd.Timestamp("1899-12-30"))
    
    #时间列太多
    df.set_index(["时间"],inplace=True)
    #交易金额前面血药加上前缀
    df.columns = [t + "交易金额"]
    #把所有的数据放入到一个list中
    dfs.append(df)

In [ ]:
df = pd.concat(dfs,axis=1)

# 观察数据

- 数据类型
- 空值
- 趋势 : 有季节因素的影响

In [ ]:
df.isnull().sum().reset_index().T

In [ ]:
df.dtypes.reset_index().T

In [ ]:
df.plot()

# 1.预测数据

In [ ]:
from sklearn.linear_model import LinearRegression  #线性回归

In [ ]:
#第一个 需要拆解时间 
#第二个 拆列数
df.head(3)

In [ ]:
def predict(df=None, year=None ,month=None):
    df = df.reset_index() 
    df["年份"] = df["时间"].dt.year
    df["月份"] = df["时间"].dt.month
    
    #预测的数据是 11月份  用往年的11月数据 预测 未来11数据
    # X一般代表 的是 预测的维度  y结果
    X_train = df.query(f"月份 == {month}")[["年份","月份"]] # 3
    y_train = df.query(f"月份 == {month}").iloc[:,1:-2]     # 3
    
    #预测的时间
    X_pred = np.array([[year,month]])
    
    #拼接词汇表时间格式
    y_index = [datetime(year,month,1)]  #["2018-11-01",1,2,3,4,5,6,7]
    
    for i in range(y_train.columns.size):
        y_tra = y_train.iloc[:,i]  #一列
        
        model = LinearRegression().fit(X_train,y_tra) #预测 会记录 系数 和偏置项
        
        #越策的结果
        y_pred = model.predict(X_pred)   #给时间  返回 金额值
        
        y_index.append(y_pred[0]) #list 
    df.drop(columns=["年份","月份"],inplace=True)
    new_df = pd.DataFrame(pd.Series(y_index,index=df.columns)).T
    
   
    
    return  pd.concat([new_df,df]).set_index("时间")

In [ ]:
#先预测 2018 11 和 12 月份数据
df = predict(df,year=2018,month=11)
df = predict(df,year=2018,month=12)

In [ ]:
# 2019 - 2027年的数据
year = range(2019,2022)
month = range(1,13)

for y in year:
    for m in month:
        df = predict(df,year=y,month=m)

In [ ]:
for m in range(1,11):
    df = predict(df,year=2015,month=m)

In [ ]:
df.plot()

# 2.市场容量

- 整个灭鼠杀虫大市场

In [ ]:
dfgp = df.reset_index().groupby(by=[df.reset_index().时间.dt.year])[df.columns].sum()

In [ ]:
dfgp["总额"] = dfgp.sum(axis=1)

In [ ]:
dfgp.style.format("￥{:,.2f}",na_rep="-")

xinds = range(dfgp.index.size)

plt.bar(xinds,dfgp.总额)

plt.xticks(xinds,dfgp.index)

#把增长率放到 注释中

for a,b in list(zip(xinds,list(zip(dfgp.总额,dfgp.总额.pct_change().fillna(0) )))):
    plt.text(a-0.35,b[0],s=f"同比增长率:{round(b[1]*100,2)}%\n年销售额:{round(b[0],2)}")
    
plt.title("2015~2021灭鼠杀虫市场容量和增长率")

如果企业的增幅 低于 GDP 说明公司业绩在下降
如果企业的增幅 持平 GDP 说明公司业绩持平
企业的增幅速度 必须 要大于GDP 才是健康状态

市场的份额逐渐扩大,增长率逐步减缓 ,说明增量市场 逐步转化 存量市场

# 3.细分市场的趋势

In [ ]:
dfgp.iloc[:,:-1].plot(marker="o")

In [ ]:
xinds = range(dfgp.shape[0])  #行数
colors = sns.color_palette("rainbow",dfgp.shape[0])
#局部调节
plt.figure(figsize=(20*2,10*4))  #第一个参数是列  第二参数是行
#多图型绘制方法
i=1
for c in dfgp.columns[:-1]:
    #第一个参数是行数   第二个参数是列数  ,第三个参数是画布的编号(必须从1开,编号不能一致)
    axes = plt.subplot(4,2,i)  #返回的是一个画布对象
    data = dfgp.reset_index().loc[:,["时间",c]]
    
    
    sns.barplot(data=data,x="时间",y=c,ax=axes,color=colors[i-1])
    sns.pointplot(data=data,x="时间",y=c,ax=axes)
    
    axes.set_title(c)
    
    for a,b in list(zip(xinds,list(zip(dfgp[c],dfgp[c].pct_change().fillna(0) )))):
        plt.text(a-0.35,b[0],s=f"同比增长率:{round(b[1]*100,2)}%",fontsize=20)
    
    i+=1
    
plt.savefig("./1.png")

# 4.细分市场的占比

In [ ]:
dfgp.iloc[:,:-1].div(dfgp.iloc[:,-1],axis=0)

In [ ]:
dfgp.iloc[:,:-1].div(dfgp.iloc[:,-1],axis=0).plot(kind="bar",stacked=True)

在各年度中 , 不同产品  在整体市场 占有份额比例情况

# 5.市场的竞争度

In [ ]:
top100 = pd.read_excel("./top100品牌数据.xlsx").iloc[:,:-1]

In [ ]:
#数据银行 : 交易指数 销售额  销售量  点击数  收藏量 .....
#交易指数 指数越大,说明 当前 产品的影响力 越大
top100["交易指数比"] = top100.交易指数.div(top100.交易指数.sum())  #总额百分比

In [ ]:
top100["累计交易指数比"] = top100["交易指数比"].cumsum()

In [ ]:

top100.query(f"交易指数比 >={top100.交易指数比.quantile(0.8)}")

In [ ]:
plt.figure(figsize=(20,6))
axes1 = plt.gca()
sns.barplot(top100,x="品牌",y="交易指数比",ax=axes1,color="skyblue")
axes2 = axes1.twinx()
sns.pointplot(top100,x="品牌",y="累计交易指数比",ax=axes2,color="orange")
# axes1.set_xticks()
a = axes1.set_xticklabels(top100.品牌,rotation=90)

axes1.vlines(19+0.5,0,0.035,ls="--")

#累计图像当中会出现比较明显的 拐点

In [ ]:
HHI = top100.交易指数比.pow(2).sum()

In [ ]:
HHI

## 赫芬达尔赫希曼指数

- $\Sigma S^2$
- S : 市场占有份额(总额百分比)
- HHI :
    - 小于0.01 : 高度自由竞争
    - 小于0.15 : 垄断不集中
    - 小于0.25 : 垄断高度集中
    - 小于0.4 : 高度垄断

假设现在有100公司 期中有一家占据89%的份额 其它的99家平分11%的份额
假设现在有100公司 期中有一家占据50%的份额 其它的99家平分50%的份额
假设现在有100公司 期中有一家占据30%的份额 其它的99家平分70%的份额

In [ ]:
.89**2 + (0.11/99)**2 * 99

In [ ]:
.5**2 + (0.5/99)**2 * 99

In [ ]:
.3**2 + (0.7/99)**2 * 99